In [1]:
import cv2 as cv
import mediapipe as mp
import numpy as np
import time
import pandas as pd
import tensorflow as tf
mp_draw = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
dataset = pd.read_csv('data_set.csv')

preprocessing 

In [3]:
dataset

,name,left_elbow_angle,right_elbow_angle,left_shoulder_angle,left_knee_angle,left_hip_angle
0,0,171.491297,169.950651,161.695863,180.244157,77.763532
1,0,158.559441,162.614523,170.457119,175.729888,78.284543
2,0,168.009165,162.824096,184.815624,167.309210,79.099682
3,0,167.148221,165.495074,171.609069,187.513810,80.763860
4,0,167.891482,172.431216,170.753887,181.224083,66.801409
...,...,...,...,...,...,...
8113,1,55.000000,74.000000,65.000000,102.000000,10.000000
8114,1,10.000000,113.000000,10.000000,13.000000,27.000000
8115,1,70.000000,128.000000,75.000000,100.000000,53.000000
8116,1,138.000000,106.000000,73.000000,75.000000,59.000000


In [4]:
y = dataset.iloc[:, 0]
X = dataset.iloc[:, 1:]

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

In [6]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

Deep Learning 

In [7]:
ann = tf.keras.models.Sequential()


In [8]:
ann.add(tf.keras.layers.Dense(units=16, activation='relu'))
ann.add(tf.keras.layers.Dense(units=32, activation='relu'))
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [9]:
ann.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [10]:
type(X_train)

numpy.ndarray

In [ ]:
ann.fit(X_train,y_train, batch_size=10, epochs=30, shuffle=True)

Epoch 1/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.9991 - loss: 0.1380
Epoch 2/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 7.2231e-04
Epoch 3/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.8837e-04
Epoch 4/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 7.8572e-05
Epoch 5/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 3.8647e-05
Epoch 6/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 2.4374e-05
Epoch 7/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 1.4375e-05
Epoch 8/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 9.9165e-06
Epoch 9/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 5.9857e-06
Epoch 10/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 3.8208e-06
Epoch 11/30
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 1.0000 - loss: 2.1963e-06
Epoch 12/30


## Angle calculating function

In [12]:
def calculateAngle(a,b,c):
    a = [a.x,a.y]
    b = [b.x,b.y]
    c = [c.x,c.y]
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radian = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radian*180.0/np.pi)

    if angle>180.0:
        angle = 360-angle
    return angle

## function to get an array of angles

In [ ]:
def angles_finder(landmarks):
    
    
    left_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value])
    
 
    right_elbow_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value])   
    
  
    left_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                         landmarks[mp_pose.PoseLandmark.LEFT_HIP.value])

 
    right_shoulder_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                          landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])


    left_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value],
                                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])


    right_knee_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    
    hand_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                      landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value])
    
    left_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value])
    
    right_hip_angle = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value])
    
    neck_angle_uk = calculateAngle(landmarks[mp_pose.PoseLandmark.NOSE.value],
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value],
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value])
    
    left_wrist_angle_bk = calculateAngle(landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_HIP.value],
                                      landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value])
    

    right_wrist_angle_bk = calculateAngle(landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value],
                                       landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value])
    #----------------------------------------------------------------------------------------------------------------
    return [left_elbow_angle,right_elbow_angle,left_shoulder_angle,left_knee_angle,left_hip_angle,]

test

In [22]:
print(ann.predict(sc.transform(X_test))>0.5)
print(y_test)

51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step
[[ True]
 [ True]
 [ True]
 ...
 [ True]
 [ True]
 [ True]]
3290    1
5234    0
5298    0
1768    0
763     0
       ..
94      0
4471    0
1610    0
5308    0
5448    0
Name: name, Length: 1624, dtype: int64


In [17]:
from sklearn.preprocessing import StandardScaler


## Please read 
### downward dog pose detection 
This project is made to detect that a person is doing downward dog pose or not with mediapipe and cv2 for the computer vision and for pose-estimation to get landmarks
To train this deep learning model the data set I used is extended by me. I was not able to find a proper data set and the dataset was not large enough. Therefore, some data I took from kagel the rest of the dataset I created using the min and max range. Also, the laptop I used to make this is not that capable therefore on applying the dl model it crashed.
Therefore, I am submitting this without being able to test this.


In [ ]:
cap = cv.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    pTime = 0
    while True:
        ret, frame = cap.read()


    #for detection and rendering

        # recolor image to RGB
        image = cv.cvtColor(frame,cv.COLOR_BGR2RGB)
        image.flags.writeable = False

        # make detections(stores it in results)
        results = pose.process(image)

        # recolor back to BGR
        image.flags.writeable = True
        image = cv.cvtColor(image, cv.COLOR_RGB2BGR)

         
        #finding landmarks
        landmarks = results.pose_landmarks.landmark
        """print(landmarks)"""

        
        
        # render detections
        mp_draw.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                               mp_draw.DrawingSpec(color=(245,117,66), thickness=1, circle_radius=2),
                               mp_draw.DrawingSpec(color=(100,100,100), thickness=2, circle_radius=2)
                               )

    #  print(results)
        # for the fps
        cTime = time.time()
        fps = 1/(cTime-pTime)
        pTime = cTime
        image = cv.flip(image,1)
        sc = StandardScaler()
        landmarks = sc.fit_transform([angles_finder(landmarks)])
        final_pred = ann.predict(landmarks)
        
        if not final_pred:
            print('good')
        else:
            print('bad')
        cv.putText(image, str(int(fps)),(70,50), cv.FONT_HERSHEY_PLAIN, 3, (255,255,255))
        cv.imshow("Mediapipe Feed", image)
        if cv.waitKey(10) == ord('q'):
            break
    cap.release()
    cv.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
bad
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
bad


In [ ]:
final_pred

array([[0.99997866]], dtype=float32)